# Tangelo Velocity Tutorial: Stage 1 Regulatory Model

This notebook demonstrates **Tangelo Velocity Stage 1** - a regulatory network model for RNA velocity estimation that integrates chromatin accessibility (ATAC-seq) data with RNA splicing dynamics.

## Overview

**Stage 1** focuses on regulatory modeling and includes:
- **Sigmoid Feature Transformation**: Learnable smooth feature mapping for RNA expression
- **Linear Interaction Networks**: Gene regulatory interactions with ATAC masking
- **ODE Dynamics**: Cell-specific splicing and degradation rate modeling
- **Regulatory Loss Functions**: Reconstruction and regulatory constraint losses

**What's Implemented (Stage 0 + Stage 1):**
- Stage 0: MuDataProcessor, GraphBuilder, Node2VecEmbedding
- Stage 1: SigmoidFeatureModule, LinearInteractionNetwork, VelocityODE, Stage1RegulatoryModel

**Future Stages (not yet implemented):**
- Stage 2: Graph neural networks for spatial modeling  
- Stage 3: Integrated multi-modal architecture
- Stage 4: Advanced features and hierarchical modeling

## Setup and Imports

In [ ]:
# Core imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Deep learning
import torch
import torch.nn as nn

# Single-cell analysis 
import scanpy as sc
try:
    import muon as mu
    import anndata as ad
    HAS_MUON = True
except ImportError:
    print("Note: muon not available. Using synthetic data for demonstration.")
    HAS_MUON = False

# Tangelo Velocity - Stage 1 components
import tangelo_velocity as tv
from tangelo_velocity.config import TangeloConfig, get_stage_config

# Configure visualization
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=100, facecolor='white', figsize=(6, 6))
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['figure.dpi'] = 100

print(f"Tangelo Velocity version: {tv.__version__}")
print(f"Available modules: {[x for x in tv.__all__ if not x.startswith('_')]}")
print(f"PyTorch version: {torch.__version__}")
print(f"Has GPU: {torch.cuda.is_available()}")

## 1. Stage 1 Configuration

Stage 1 focuses on regulatory modeling. Let's start by exploring the configuration options:

In [ ]:
# Get default Stage 1 configuration
stage1_config = get_stage_config(stage=1)

print("=== Stage 1 Configuration Overview ===")
print(f"Development Stage: {stage1_config.development_stage}")
print(f"Gene dimension: {stage1_config.gene_dim} (will be inferred from data)")
print(f"ATAC dimension: {stage1_config.atac_dim} (will be inferred from data)")

print("\n=== Regulatory Configuration ===")
print(f"Use ATAC masking: {stage1_config.regulatory.use_atac_masking}")
print(f"ATAC threshold: {stage1_config.regulatory.atac_threshold}")
print(f"Use sigmoid features: {stage1_config.regulatory.use_sigmoid_features}")
print(f"Sigmoid components: {stage1_config.regulatory.n_sigmoid_components}")
print(f"Interaction strength: {stage1_config.regulatory.interaction_strength}")

print("\n=== ODE Configuration ===")
print(f"Solver: {stage1_config.ode.solver}")
print(f"Tolerance (rtol/atol): {stage1_config.ode.rtol}/{stage1_config.ode.atol}")
print(f"Time span: {stage1_config.ode.t_span}")
print(f"Beta range: {stage1_config.ode.init_beta_range}")
print(f"Gamma range: {stage1_config.ode.init_gamma_range}")

print("\n=== Training Configuration ===")
print(f"Epochs: {stage1_config.training.n_epochs}")
print(f"Learning rate: {stage1_config.training.learning_rate}")
print(f"Batch size: {stage1_config.training.batch_size}")
print(f"Optimizer: {stage1_config.training.optimizer}")

print("\n=== Loss Configuration ===")
print(f"Reconstruction weight: {stage1_config.loss.reconstruction_weight}")
print(f"L2 regularization: {stage1_config.loss.l2_reg}")

In [ ]:
def create_synthetic_stage1_data(n_cells=500, n_genes=200, n_peaks=300):
    """
    Create synthetic multi-modal data for Stage 1 demonstration.
    
    This creates realistic RNA and ATAC data with regulatory relationships
    suitable for testing the Stage 1 regulatory model.
    """
    print(f"Creating synthetic Stage 1 data: {n_cells} cells, {n_genes} genes, {n_peaks} peaks")
    
    # Set random seed for reproducibility
    np.random.seed(42)
    torch.manual_seed(42)
    
    # Create realistic RNA expression data
    # Simulate multiple cell types with different expression programs
    n_programs = 3
    program_genes = n_genes // n_programs
    
    # Base expression levels
    base_expression = np.random.gamma(2, 2, (n_cells, n_genes))
    
    # Add cell type specific programs
    cell_types = np.random.choice(n_programs, n_cells)
    for prog in range(n_programs):
        mask = cell_types == prog
        gene_start = prog * program_genes
        gene_end = (prog + 1) * program_genes
        base_expression[mask, gene_start:gene_end] *= 3  # Upregulate program genes
    
    # Create spliced/unspliced from base expression
    # Simulate splicing dynamics: more unspliced in highly expressed genes
    splicing_efficiency = np.random.beta(2, 1, n_genes)  # Varies by gene
    
    unspliced = base_expression * (1 - splicing_efficiency) + np.random.gamma(1, 0.5, (n_cells, n_genes))
    spliced = base_expression * splicing_efficiency + np.random.gamma(1, 0.5, (n_cells, n_genes))
    
    # Simulate chromatin accessibility 
    # Create gene-peak linkages (regulatory relationships)
    peak_gene_links = np.random.binomial(1, 0.3, (n_peaks, n_genes))  # 30% linkage probability
    
    # ATAC signal correlates with linked gene expression
    atac_data = np.zeros((n_cells, n_peaks))
    for peak in range(n_peaks):
        linked_genes = peak_gene_links[peak, :] > 0
        if linked_genes.sum() > 0:
            # Peak accessibility correlates with linked gene expression
            atac_data[:, peak] = spliced[:, linked_genes].mean(axis=1) * 0.5
        # Add noise
        atac_data[:, peak] += np.random.gamma(0.5, 1, n_cells)
    
    # Simulate open chromatin (binary accessibility for regulatory masking)
    open_chromatin = (atac_data > np.percentile(atac_data, 70, axis=0)).astype(float)
    
    # Create gene-specific open chromatin signal
    gene_chromatin = np.zeros((n_cells, n_genes))
    for gene in range(n_genes):
        linked_peaks = peak_gene_links[:, gene] > 0
        if linked_peaks.sum() > 0:
            gene_chromatin[:, gene] = open_chromatin[:, linked_peaks].mean(axis=1)
        else:
            gene_chromatin[:, gene] = np.random.binomial(1, 0.2, n_cells)  # Background accessibility
    
    # Create moments for velocity analysis
    M_s = spliced + np.random.normal(0, 0.1 * spliced)
    M_u = unspliced + np.random.normal(0, 0.1 * unspliced)
    
    print(f"Created synthetic data with:")
    print(f"  - {n_programs} cell type programs")
    print(f"  - {peak_gene_links.sum()} peak-gene regulatory links")
    print(f"  - {(gene_chromatin > 0.5).sum()} accessible gene-peak pairs")
    
    return {
        'spliced': spliced.astype(np.float32),
        'unspliced': unspliced.astype(np.float32), 
        'M_s': M_s.astype(np.float32),
        'M_u': M_u.astype(np.float32),
        'gene_chromatin': gene_chromatin.astype(np.float32),
        'atac_data': atac_data.astype(np.float32),
        'open_chromatin': open_chromatin.astype(np.float32),
        'cell_types': cell_types,
        'peak_gene_links': peak_gene_links.astype(np.float32)
    }

# Create synthetic data for Stage 1 demo
data = create_synthetic_stage1_data(n_cells=500, n_genes=200, n_peaks=300)

In [ ]:
# Visualize the synthetic data
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# RNA expression by cell type
for i, cell_type in enumerate(np.unique(data['cell_types'])):
    mask = data['cell_types'] == cell_type
    axes[0, 0].scatter(np.log1p(data['spliced'][mask].mean(axis=1)), 
                      np.log1p(data['unspliced'][mask].mean(axis=1)),
                      label=f'Type {cell_type}', alpha=0.6, s=20)
axes[0, 0].set_xlabel('Log(Spliced + 1)')
axes[0, 0].set_ylabel('Log(Unspliced + 1)')
axes[0, 0].set_title('RNA Expression by Cell Type')
axes[0, 0].legend()

# Chromatin accessibility distribution
axes[0, 1].hist(data['gene_chromatin'].mean(axis=0), bins=30, alpha=0.7, 
               label='Gene accessibility', color='orange')
axes[0, 1].set_xlabel('Mean Accessibility')
axes[0, 1].set_ylabel('Number of Genes')
axes[0, 1].set_title('Chromatin Accessibility Distribution')

# Expression vs Accessibility correlation
expr_mean = data['spliced'].mean(axis=0)
chrom_mean = data['gene_chromatin'].mean(axis=0)
correlation = np.corrcoef(expr_mean, chrom_mean)[0, 1]
axes[0, 2].scatter(expr_mean, chrom_mean, alpha=0.6, s=20)
axes[0, 2].set_xlabel('Mean Spliced Expression')
axes[0, 2].set_ylabel('Mean Chromatin Accessibility')
axes[0, 2].set_title(f'Expression vs Accessibility\n(r = {correlation:.3f})')

# Splicing efficiency (spliced / (spliced + unspliced))
splicing_eff = data['spliced'] / (data['spliced'] + data['unspliced'] + 1e-6)
axes[1, 0].hist(splicing_eff.mean(axis=0), bins=30, alpha=0.7, color='green')
axes[1, 0].set_xlabel('Mean Splicing Efficiency')
axes[1, 0].set_ylabel('Number of Genes')
axes[1, 0].set_title('Splicing Efficiency Distribution')

# Peak-gene regulatory network
axes[1, 1].imshow(data['peak_gene_links'][:50, :50], cmap='Blues', aspect='auto')
axes[1, 1].set_xlabel('Genes (first 50)')
axes[1, 1].set_ylabel('Peaks (first 50)')
axes[1, 1].set_title('Peak-Gene Regulatory Links')

# Cell type composition
cell_counts = np.bincount(data['cell_types'])
axes[1, 2].pie(cell_counts, labels=[f'Type {i}' for i in range(len(cell_counts))], 
              autopct='%1.1f%%', startangle=90)
axes[1, 2].set_title('Cell Type Composition')

plt.tight_layout()
plt.show()

print("=== Data Summary ===")
print(f"RNA data shape: {data['spliced'].shape}")
print(f"ATAC data shape: {data['atac_data'].shape}")
print(f"Gene chromatin shape: {data['gene_chromatin'].shape}")
print(f"Regulatory links: {data['peak_gene_links'].sum():.0f}")
print(f"Mean expression: {data['spliced'].mean():.2f}")
print(f"Mean accessibility: {data['gene_chromatin'].mean():.3f}")

## 2. Stage 1 Component Demonstration

Let's explore the individual Stage 1 components before building the full model:

In [ ]:
# Import Stage 1 specific components
if hasattr(tv, 'models'):
    from tangelo_velocity.models import (
        SigmoidFeatureModule, 
        LinearInteractionNetwork,
        VelocityODE,
        ODEParameterPredictor,
        ReconstructionLoss,
        RegulatoryNetworkLoss
    )
    
    print("=== Stage 1 Components Available ===")
    print("✓ SigmoidFeatureModule - Learnable feature transformation")
    print("✓ LinearInteractionNetwork - Gene regulatory interactions") 
    print("✓ VelocityODE - ODE dynamics modeling")
    print("✓ ODEParameterPredictor - Cell-specific parameter prediction")
    print("✓ ReconstructionLoss - RNA abundance matching")
    print("✓ RegulatoryNetworkLoss - Regulatory constraint loss")
    
else:
    print("Models module not available - dependencies may be missing")
    print("This tutorial will demonstrate configuration and concepts only")

# Convert data to tensors for component testing
n_cells, n_genes = data['spliced'].shape
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

spliced_tensor = torch.tensor(data['spliced'], dtype=torch.float32, device=device)
unspliced_tensor = torch.tensor(data['unspliced'], dtype=torch.float32, device=device) 
chromatin_tensor = torch.tensor(data['gene_chromatin'], dtype=torch.float32, device=device)

print(f"\nData loaded on device: {device}")
print(f"Tensor shapes - Spliced: {spliced_tensor.shape}, Chromatin: {chromatin_tensor.shape}")

### 2.1 Sigmoid Feature Module

The `SigmoidFeatureModule` provides learnable smooth transformations of RNA expression data:

In [ ]:
if hasattr(tv, 'models'):
    # Create sigmoid feature module
    sigmoid_module = SigmoidFeatureModule(
        n_genes=n_genes,
        n_components=10,  # Number of sigmoid components per gene
        init_a=1.0,       # Initial slope
        init_b=0.0        # Initial bias
    ).to(device)
    
    print(f"Sigmoid module parameters: {sum(p.numel() for p in sigmoid_module.parameters())}")
    
    # Apply transformation to spliced data
    transformed_features = sigmoid_module(spliced_tensor)
    
    print(f"Input shape: {spliced_tensor.shape}")
    print(f"Output shape: {transformed_features.shape}")
    print(f"Input range: [{spliced_tensor.min():.3f}, {spliced_tensor.max():.3f}]")
    print(f"Output range: [{transformed_features.min():.3f}, {transformed_features.max():.3f}]")
    
    # Visualize transformation for first few genes
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    for i in range(3):
        x_vals = spliced_tensor[:, i].cpu().numpy()
        y_vals = transformed_features[:, i].detach().cpu().numpy()
        
        # Sort for visualization
        sort_idx = np.argsort(x_vals)
        x_sorted = x_vals[sort_idx]
        y_sorted = y_vals[sort_idx]
        
        axes[i].scatter(x_vals, y_vals, alpha=0.5, s=10, label='Data points')
        axes[i].plot(x_sorted, y_sorted, 'r-', alpha=0.8, label='Transformation')
        axes[i].set_xlabel(f'Original Expression (Gene {i})')
        axes[i].set_ylabel(f'Transformed Expression')
        axes[i].set_title(f'Sigmoid Transformation - Gene {i}')
        axes[i].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Show learnable parameters for first gene
    params = sigmoid_module.get_parameters()
    print(f"\\nParameters for Gene 0:")
    print(f"Slopes: {params['slopes'][0].cpu().numpy()[:5]}")  # First 5 components
    print(f"Biases: {params['biases'][0].cpu().numpy()[:5]}")
    print(f"Weights: {params['weights'][0].cpu().numpy()[:5]}")
    
else:
    print("Sigmoid feature demonstration requires pytorch and model components")

In [ ]:
### 2.2 Linear Interaction Network with ATAC Masking

The `LinearInteractionNetwork` models gene regulatory interactions with chromatin accessibility constraints:

if hasattr(tv, 'models'):
    # Create linear interaction network
    interaction_net = LinearInteractionNetwork(
        n_genes=n_genes,
        use_bias=False,           # No bias terms
        interaction_strength=1.0  # Scaling factor
    ).to(device)
    
    print(f"Interaction network parameters: {sum(p.numel() for p in interaction_net.parameters())}")
    
    # Test without ATAC masking
    interactions_no_mask = interaction_net(transformed_features)
    
    # Test with ATAC masking
    interactions_with_mask = interaction_net(transformed_features, atac_mask=chromatin_tensor)
    
    print(f"\\n=== Interaction Network Results ===")
    print(f"Input features shape: {transformed_features.shape}")
    print(f"ATAC mask shape: {chromatin_tensor.shape}")
    print(f"Output without masking: {interactions_no_mask.shape}")
    print(f"Output with masking: {interactions_with_mask.shape}")
    
    # Compare interaction strengths
    no_mask_norm = torch.norm(interactions_no_mask, dim=1).mean()
    with_mask_norm = torch.norm(interactions_with_mask, dim=1).mean()
    
    print(f"\\nMean interaction strength:")
    print(f"  Without ATAC masking: {no_mask_norm:.4f}")
    print(f"  With ATAC masking: {with_mask_norm:.4f}")
    print(f"  Masking effect: {(with_mask_norm/no_mask_norm):.4f}x")
    
    # Visualize interaction matrix and masking effect
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Raw interaction matrix (first 20x20 genes)
    W = interaction_net.W.detach().cpu().numpy()[:20, :20]
    im1 = axes[0].imshow(W, cmap='RdBu_r', center=0)
    axes[0].set_title('Raw Interaction Matrix\\n(first 20x20 genes)')
    axes[0].set_xlabel('Gene Index')
    axes[0].set_ylabel('Gene Index')
    plt.colorbar(im1, ax=axes[0])
    
    # ATAC masking pattern (first 20 genes, first 50 cells)
    mask_pattern = chromatin_tensor[:50, :20].cpu().numpy()
    im2 = axes[1].imshow(mask_pattern.T, cmap='Oranges', aspect='auto')
    axes[1].set_title('ATAC Masking Pattern\\n(first 20 genes, 50 cells)')
    axes[1].set_xlabel('Cell Index')
    axes[1].set_ylabel('Gene Index')
    plt.colorbar(im2, ax=axes[1])
    
    # Effect of masking on interactions
    diff = (interactions_with_mask - interactions_no_mask)[:50, :20].detach().cpu().numpy()
    im3 = axes[2].imshow(diff.T, cmap='RdBu_r', center=0, aspect='auto')
    axes[2].set_title('Masking Effect on Interactions\\n(first 20 genes, 50 cells)')
    axes[2].set_xlabel('Cell Index')
    axes[2].set_ylabel('Gene Index')
    plt.colorbar(im3, ax=axes[2])
    
    plt.tight_layout()
    plt.show()
    
    # Show statistics
    print(f"\\n=== Interaction Matrix Statistics ===")
    print(f"Interaction matrix shape: {W.shape}")
    print(f"Non-zero interactions: {(np.abs(W) > 1e-6).sum()}/{W.size}")
    print(f"Mean absolute interaction: {np.abs(W).mean():.6f}")
    print(f"Max interaction strength: {np.abs(W).max():.6f}")
    
else:
    print("Interaction network demonstration requires model components")

In [ ]:
### 2.3 ODE Dynamics and Parameter Prediction

The `VelocityODE` models RNA dynamics with cell-specific parameters:

In [ ]:
if hasattr(tv, 'models'):
    # Create ODE parameter predictor
    param_predictor = ODEParameterPredictor(
        input_dim=n_genes,         # Uses regulatory interactions as input
        hidden_dim=64,             # Hidden layer size
        init_beta_range=(0.1, 2.0), # Splicing rate range
        init_gamma_range=(0.1, 1.0) # Degradation rate range
    ).to(device)
    
    # Predict cell-specific ODE parameters from regulatory interactions
    ode_params = param_predictor(interactions_with_mask)
    
    print(f"=== ODE Parameter Prediction ===")
    print(f"Input (regulatory interactions): {interactions_with_mask.shape}")
    print(f"Predicted parameters: {ode_params.keys()}")
    
    for param_name, param_tensor in ode_params.items():
        print(f"  {param_name}: {param_tensor.shape}, range [{param_tensor.min():.4f}, {param_tensor.max():.4f}]")
    
    # Create ODE dynamics module
    ode_dynamics = VelocityODE(stage1_config.ode).to(device)
    
    # Simulate ODE dynamics with predicted parameters
    t_span = torch.tensor([0.0, 1.0], device=device)
    initial_state = torch.stack([unspliced_tensor, spliced_tensor], dim=-1)  # (cells, genes, 2)
    
    print(f"\\n=== ODE Integration ===")
    print(f"Time span: {t_span}")
    print(f"Initial state shape: {initial_state.shape}")
    print(f"Initial unspliced range: [{unspliced_tensor.min():.3f}, {unspliced_tensor.max():.3f}]")
    print(f"Initial spliced range: [{spliced_tensor.min():.3f}, {spliced_tensor.max():.3f}]")
    
    # Visualize ODE parameters across cells and genes
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Beta (splicing rate) distribution
    beta_vals = ode_params['beta'].detach().cpu().numpy()
    axes[0].hist(beta_vals.flatten(), bins=30, alpha=0.7, color='blue', edgecolor='black')
    axes[0].set_xlabel('Splicing Rate (β)')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title(f'Splicing Rate Distribution\\nMean: {beta_vals.mean():.3f} ± {beta_vals.std():.3f}')
    axes[0].axvline(beta_vals.mean(), color='red', linestyle='--', label='Mean')
    axes[0].legend()
    
    # Gamma (degradation rate) distribution
    gamma_vals = ode_params['gamma'].detach().cpu().numpy()
    axes[1].hist(gamma_vals.flatten(), bins=30, alpha=0.7, color='green', edgecolor='black')
    axes[1].set_xlabel('Degradation Rate (γ)')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title(f'Degradation Rate Distribution\\nMean: {gamma_vals.mean():.3f} ± {gamma_vals.std():.3f}')
    axes[1].axvline(gamma_vals.mean(), color='red', linestyle='--', label='Mean')
    axes[1].legend()
    
    # Time parameter distribution
    t_vals = ode_params['t'].detach().cpu().numpy()
    axes[2].hist(t_vals.flatten(), bins=30, alpha=0.7, color='orange', edgecolor='black')
    axes[2].set_xlabel('Cell Time (t)')
    axes[2].set_ylabel('Frequency')
    axes[2].set_title(f'Cell Time Distribution\\nMean: {t_vals.mean():.3f} ± {t_vals.std():.3f}')
    axes[2].axvline(t_vals.mean(), color='red', linestyle='--', label='Mean')
    axes[2].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Show correlation between parameters and regulatory interactions
    interaction_strength = torch.norm(interactions_with_mask, dim=1).detach().cpu().numpy()
    beta_mean = beta_vals.mean(axis=1)
    gamma_mean = gamma_vals.mean(axis=1)
    
    print(f"\\n=== Parameter Correlations ===")
    beta_corr = np.corrcoef(interaction_strength, beta_mean)[0, 1]
    gamma_corr = np.corrcoef(interaction_strength, gamma_mean)[0, 1]
    print(f"Interaction strength vs β: r = {beta_corr:.3f}")
    print(f"Interaction strength vs γ: r = {gamma_corr:.3f}")
    
else:
    print("ODE dynamics demonstration requires model components")

## 3. Full Stage 1 Model Integration

While the individual components are demonstrated above, the full Stage 1 model integrates all components. However, the high-level API (`TangeloVelocity`) is currently being developed and requires additional dependencies.

In [ ]:
# Check if Stage 1 model is available
if hasattr(tv, 'models'):
    try:
        from tangelo_velocity.models import Stage1RegulatoryModel, get_velocity_model
        
        print("=== Stage 1 Model Architecture ===")
        print("The Stage1RegulatoryModel integrates:")
        print("1. SigmoidFeatureModule for smooth expression transformation")
        print("2. LinearInteractionNetwork with ATAC masking")
        print("3. ODEParameterPredictor for cell-specific dynamics")
        print("4. VelocityODE for RNA dynamics simulation")
        print("5. Specialized loss functions (ReconstructionLoss + RegulatoryLoss)")
        
        # Create a Stage 1 model configuration
        stage1_config.gene_dim = n_genes
        stage1_config.atac_dim = n_genes  # Using gene-linked chromatin
        
        print(f"\\nModel configuration:")
        print(f"  Stage: {stage1_config.development_stage}")
        print(f"  Gene dimension: {stage1_config.gene_dim}")
        print(f"  ATAC dimension: {stage1_config.atac_dim}")
        print(f"  Device: {stage1_config.device}")
        
        # For a complete example, you would instantiate and train the model:
        print(f"\\n=== Model Usage Example ===")
        print("# model = get_velocity_model(stage1_config, n_genes, n_genes)")
        print("# model = model.to(device)")
        print("# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)")
        print("# loss = model.compute_loss(outputs, targets)")
        print("# loss.backward()")
        print("# optimizer.step()")
        
        print(f"\\n✓ Stage 1 components are ready for integration")
        
    except Exception as e:
        print(f"Stage 1 model integration note: {e}")
        print("All individual components demonstrated above are functional")
        
else:
    print("Stage 1 model demonstration requires dependency installation")
    print("Individual components shown above demonstrate the core functionality")

In [ ]:
## 4. Stage 1 Configuration Customization

You can customize Stage 1 behavior through configuration:

# Create custom Stage 1 configurations for different use cases

# Configuration 1: High regulatory constraints
high_regulatory_config = TangeloConfig(
    development_stage=1,
    regulatory=tv.config.RegulatoryConfig(
        use_atac_masking=True,
        atac_threshold=0.2,  # Stricter ATAC threshold
        use_sigmoid_features=True,
        n_sigmoid_components=20,  # More components for complex transformations
        interaction_strength=2.0  # Stronger regulatory interactions
    ),
    ode=tv.config.ODEConfig(
        solver="dopri5",
        rtol=1e-6,  # Higher precision
        atol=1e-8,
        init_beta_range=(0.5, 3.0),  # Wider splicing rate range
        init_gamma_range=(0.1, 1.5)
    ),
    training=tv.config.TrainingConfig(
        n_epochs=150,
        learning_rate=5e-4,  # Slower learning for stability
        batch_size=256
    )
)

# Configuration 2: Fast training for exploration
fast_config = TangeloConfig(
    development_stage=1, 
    regulatory=tv.config.RegulatoryConfig(
        use_atac_masking=False,  # Disable for speed
        use_sigmoid_features=True,
        n_sigmoid_components=5,  # Fewer components
        interaction_strength=1.0
    ),
    training=tv.config.TrainingConfig(
        n_epochs=50,  # Quick training
        learning_rate=1e-3,
        batch_size=512  # Larger batches
    )
)

# Configuration 3: High-precision modeling
precision_config = TangeloConfig(
    development_stage=1,
    regulatory=tv.config.RegulatoryConfig(
        use_atac_masking=True,
        atac_threshold=0.05,  # Very fine-grained masking
        use_sigmoid_features=True, 
        n_sigmoid_components=30,  # Maximum detail
        interaction_strength=1.5
    ),
    ode=tv.config.ODEConfig(
        solver="dopri5",
        rtol=1e-7,  # Maximum precision
        atol=1e-9,
        max_steps=2000,  # Allow more integration steps
        n_time_points=100  # Fine temporal resolution
    ),
    loss=tv.config.LossConfig(
        reconstruction_weight=1.0,
        l2_reg=1e-5  # Light regularization
    )
)

print("=== Stage 1 Configuration Examples ===")
print(f"1. High Regulatory: {high_regulatory_config.regulatory.n_sigmoid_components} sigmoid components, " +
      f"ATAC threshold {high_regulatory_config.regulatory.atac_threshold}")
print(f"2. Fast Training: {fast_config.training.n_epochs} epochs, " +
      f"batch size {fast_config.training.batch_size}")
print(f"3. High Precision: {precision_config.ode.rtol} relative tolerance, " +
      f"{precision_config.ode.max_steps} max ODE steps")

# Save configurations for later use
high_regulatory_config.save_yaml("stage1_high_regulatory.yaml")
fast_config.save_yaml("stage1_fast_training.yaml")
precision_config.save_yaml("stage1_high_precision.yaml")

print(f"\\n✓ Configurations saved as YAML files")
print("You can load them with: TangeloConfig.from_yaml('filename.yaml')")

## Summary: Stage 1 Regulatory Model

This tutorial demonstrated **Tangelo Velocity Stage 1**, focusing on regulatory network modeling for RNA velocity estimation.

### Key Components Implemented:

1. **SigmoidFeatureModule**: Learnable smooth transformations of gene expression with multiple sigmoid components per gene
2. **LinearInteractionNetwork**: Gene regulatory interactions with ATAC-seq masking for chromatin accessibility constraints  
3. **VelocityODE**: ODE dynamics modeling with TorchODE integration for RNA splicing/degradation
4. **ODEParameterPredictor**: Cell-specific prediction of splicing (β) and degradation (γ) rates
5. **Loss Functions**: ReconstructionLoss and RegulatoryNetworkLoss for training

### Stage 1 Capabilities:

✅ **Regulatory modeling**: Integrates chromatin accessibility data  
✅ **Cell-specific dynamics**: Predicts individual ODE parameters per cell  
✅ **Flexible configuration**: Extensive customization options  
✅ **GPU acceleration**: PyTorch-based implementation with CUDA support  

### Current Implementation Status:

- **Stage 0**: ✅ Complete (preprocessing, graph construction, embeddings)
- **Stage 1**: ✅ Complete (regulatory modeling as demonstrated)
- **Stage 2**: 🚧 Planned (graph neural networks for spatial modeling)
- **Stage 3**: 🚧 Planned (integrated multi-modal architecture)  
- **Stage 4**: 🚧 Planned (advanced features and hierarchical modeling)

### Next Steps:

1. **Install dependencies** (numpy, torch, scanpy, muon) to run the interactive examples
2. **Prepare your data** in MuData format with RNA + ATAC modalities
3. **Configure Stage 1** for your specific use case (regulatory vs speed vs precision)
4. **Train the model** using the Stage1RegulatoryModel class
5. **Analyze results** using the component introspection methods

For questions or issues, see the [Tangelo Velocity GitHub repository](https://github.com/yourusername/tangelo-velocity).

In [ ]:
# Visualize stage comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, stage in enumerate([1, 2, 3]):
    velocity = stage_results[stage]['rna'].layers['velocity']
    velocity_magnitude = np.linalg.norm(velocity, axis=1)
    
    # Histogram of velocity magnitudes
    axes[i].hist(velocity_magnitude, bins=30, alpha=0.7, density=True)
    axes[i].set_xlabel('Velocity Magnitude')
    axes[i].set_ylabel('Density')
    axes[i].set_title(f'Stage {stage}\nVelocity Distribution')
    axes[i].axvline(velocity_magnitude.mean(), color='red', linestyle='--', 
                   label=f'Mean: {velocity_magnitude.mean():.3f}')
    axes[i].legend()

plt.tight_layout()
plt.show()

# Summary comparison
print("\n=== Stage Comparison Summary ===")
comparison_df = pd.DataFrame({
    'Stage': [1, 2, 3],
    'Mean_Velocity': [np.linalg.norm(stage_results[s]['rna'].layers['velocity'], axis=1).mean() 
                     for s in [1, 2, 3]],
    'Std_Velocity': [np.linalg.norm(stage_results[s]['rna'].layers['velocity'], axis=1).std() 
                    for s in [1, 2, 3]],
    'Max_Velocity': [np.linalg.norm(stage_results[s]['rna'].layers['velocity'], axis=1).max() 
                    for s in [1, 2, 3]]
})

print(comparison_df.round(4))

## 7. Configuration Management

Save and load configurations for reproducible experiments:

In [ ]:
print("=== Configuration Management ===")

# Create and save a custom configuration
custom_config = tv.TangeloConfig(
    development_stage=3,
    graph=tv.config.GraphConfig(
        n_neighbors_spatial=10,
        n_neighbors_expression=20,
        use_node2vec=True,
        node2vec_dim=64,
    ),
    encoder=tv.config.EncoderConfig(
        latent_dim=64,
        hidden_dims=(512, 256, 128),
        fusion_method="attention",
    ),
    training=tv.config.TrainingConfig(
        n_epochs=200,
        learning_rate=5e-4,
        batch_size=1024,
    )
)

# Save configuration
config_path = Path("custom_tangelo_config.yaml")
custom_config.save_yaml(config_path)
print(f"Configuration saved to: {config_path}")

# Load configuration
loaded_config = tv.TangeloConfig.from_yaml(config_path)
print(f"Configuration loaded successfully")
print(f"  - Stage: {loaded_config.development_stage}")
print(f"  - Spatial neighbors: {loaded_config.graph.n_neighbors_spatial}")
print(f"  - Use Node2Vec: {loaded_config.graph.use_node2vec}")
print(f"  - Latent dim: {loaded_config.encoder.latent_dim}")
print(f"  - Fusion method: {loaded_config.encoder.fusion_method}")

## 8. Analysis and Metrics

Evaluate velocity quality and perform trajectory analysis:

In [ ]:
print("=== Velocity Metrics and Analysis ===")

# Use the best result from stage comparison
best_result = stage_results[3]  # Stage 3 result

# Initialize velocity metrics
try:
    metrics = tv.analysis.VelocityMetrics(best_result)
    summary = metrics.summary()
    
    print("Velocity Quality Metrics:")
    for metric, value in summary.items():
        print(f"  {metric}: {value:.4f}")
        
except Exception as e:
    print(f"Metrics computation not available: {e}")
    
    # Compute basic metrics manually
    velocity = best_result['rna'].layers['velocity']
    velocity_magnitude = np.linalg.norm(velocity, axis=1)
    
    print("Basic Velocity Statistics:")
    print(f"  Mean magnitude: {velocity_magnitude.mean():.4f}")
    print(f"  Std magnitude: {velocity_magnitude.std():.4f}")
    print(f"  Min magnitude: {velocity_magnitude.min():.4f}")
    print(f"  Max magnitude: {velocity_magnitude.max():.4f}")
    print(f"  Non-zero velocities: {(velocity_magnitude > 1e-6).sum()}/{len(velocity_magnitude)}")

## 9. Saving Results

Save your velocity estimation results:

In [ ]:
print("=== Saving Results ===")

# Save the final result
output_path = Path("tangelo_velocity_results.h5mu")
best_result.write_h5mu(output_path)
print(f"Results saved to: {output_path}")

# Save velocity matrix separately
velocity_path = Path("velocity_matrix.csv")
velocity_df = pd.DataFrame(
    best_result['rna'].layers['velocity'],
    index=best_result['rna'].obs_names,
    columns=best_result['rna'].var_names
)
velocity_df.to_csv(velocity_path)
print(f"Velocity matrix saved to: {velocity_path}")

# Save cell metadata with ODE parameters
metadata_path = Path("cell_metadata.csv")
metadata_df = best_result.obs.copy()
metadata_df.to_csv(metadata_path)
print(f"Cell metadata saved to: {metadata_path}")

print("\nAll results saved successfully!")

## Summary

This tutorial demonstrated:

1. **Data preparation** for Tangelo Velocity (MuData with spatial, RNA, and ATAC modalities)
2. **Basic velocity estimation** with one-line interface
3. **Advanced API usage** with custom configurations
4. **Model component extraction** (latent representations, ODE parameters, interaction networks)
5. **Downstream analysis** (velocity graphs, embeddings, visualization)
6. **Stage comparison** across different development stages
7. **Configuration management** for reproducible experiments
8. **Quality metrics** and analysis tools
9. **Result saving** and export

## Next Steps

- **Use your own data**: Replace the synthetic data with your multi-modal dataset
- **Experiment with stages**: Try different development stages (1-4) based on your needs
- **Tune parameters**: Adjust graph construction, encoder architecture, and training parameters
- **Advanced analysis**: Explore perturbation analysis and trajectory modeling
- **Visualization**: Create publication-ready plots with the plotting module

For more information, see the [Tangelo Velocity documentation](https://github.com/yourusername/tangelo-velocity).